In [8]:
import xarray as xr
import numpy as np

# Load the dataset
ds = xr.open_dataset("D:\Groundwater Vulnerability Mapping\data\RAW Data\GLDAS_NOAH025_M_2.1-20250814_151559\GLDAS_NOAH025_M.A200001.021.nc4")  # Replace with your filename

# Get lat/lon arrays
lats = ds['lat'].values
lons = ds['lon'].values

# List of variables (excluding time_bnds which is not spatial)
variables = [var for var in ds.data_vars if set(ds[var].dims) >= {'lat', 'lon'}]

valid_points_found = 0

print(f"Searching for the first 5 valid data points...\n")

for i, lat in enumerate(lats):
    for j, lon in enumerate(lons):
        # Check if at least one variable at this lat-lon has non-NaN data
        has_valid_data = False
        for var in variables:
            val = ds[var].isel(time=0, lat=i, lon=j).values
            if not np.isnan(val):
                has_valid_data = True
                break
        
        if has_valid_data:
            print(f"\n--- Valid Point {valid_points_found + 1} ---")
            print(f"Latitude: {lat}, Longitude: {lon}")

            for var in variables:
                val = ds[var].isel(time=0, lat=i, lon=j).values
                print(f"{var}: {val}")

            valid_points_found += 1
        
        if valid_points_found >= 5:
            break
    if valid_points_found >= 5:
        break


Searching for the first 5 valid data points...


--- Valid Point 1 ---
Latitude: -55.625, Longitude: -68.125
Swnet_tavg: 127.89435577392578
Lwnet_tavg: -43.81508255004883
Qle_tavg: 45.2786750793457
Qh_tavg: 38.90801239013672
Qg_tavg: -0.17432762682437897
Snowf_tavg: 0.0
Rainf_tavg: 3.5384677175898105e-05
Evap_tavg: 1.8101132809533738e-05
Qs_acc: 0.0013320967555046082
Qsb_acc: 0.04082737863063812
Qsm_acc: 0.0
AvgSurfT_inst: 279.7496337890625
Albedo_inst: 14.0
SWE_inst: 0.0
SnowDepth_inst: 0.0
SoilMoi0_10cm_inst: 26.130611419677734
SoilMoi10_40cm_inst: 77.51742553710938
SoilMoi40_100cm_inst: 150.65809631347656
SoilMoi100_200cm_inst: 242.81396484375
SoilTMP0_10cm_inst: 279.7696838378906
SoilTMP10_40cm_inst: 279.8160705566406
SoilTMP40_100cm_inst: 279.7863464355469
SoilTMP100_200cm_inst: 279.61016845703125
PotEvap_tavg: 109.39259338378906
ECanop_tavg: 19.019193649291992
Tveg_tavg: 2.8281047344207764
ESoil_tavg: 23.450363159179688
RootMoist_inst: 497.1201477050781
CanopInt_inst: 0.247744932

In [11]:
import xarray as xr
import numpy as np
import pandas as pd

# Load dataset
file_path = 'D:\Groundwater Vulnerability Mapping\data\RAW Data\GLDAS_CLSM10_M_2.1-20250814_153125\GLDAS_CLSM10_M.A200001.021.nc4'  # Update this to your actual file path
ds = xr.open_dataset(file_path)

# Use the first time step
time_step = ds['time'][0]

# Choose one variable to find valid data locations (e.g., Surface Soil Moisture)
valid_mask = ~np.isnan(ds['SoilMoist_S_inst'].sel(time=time_step))

# Get the indices of valid (lat, lon) pairs
valid_points = np.array(np.where(valid_mask.values)).T

# If there are fewer than 5 valid points
if len(valid_points) < 5:
    raise ValueError("Not enough valid data points in the dataset.")

# Randomly select 5 valid points
np.random.seed(42)
selected_indices = valid_points[np.random.choice(len(valid_points), 5, replace=False)]

# Get lat/lon values
lats = ds['lat'].values
lons = ds['lon'].values

# Collect data
data = []
for idx in selected_indices:
    lat_idx, lon_idx = idx
    lat = lats[lat_idx]
    lon = lons[lon_idx]

    s = ds['SoilMoist_S_inst'].sel(lat=lat, lon=lon, time=time_step, method='nearest').item()
    rz = ds['SoilMoist_RZ_inst'].sel(lat=lat, lon=lon, time=time_step, method='nearest').item()
    p = ds['SoilMoist_P_inst'].sel(lat=lat, lon=lon, time=time_step, method='nearest').item()

    data.append({
        'Latitude': lat,
        'Longitude': lon,
        'SoilMoist_S_inst': s,
        'SoilMoist_RZ_inst': rz,
        'SoilMoist_P_inst': p
    })

# Display results
df = pd.DataFrame(data)
print(df)


   Latitude  Longitude  SoilMoist_S_inst  SoilMoist_RZ_inst  SoilMoist_P_inst
0      65.5     -149.5          4.591129         213.517609        842.301880
1      54.5      -84.5          6.437433         324.279907       2218.286133
2      22.5       96.5          5.582534         280.239075       1851.348877
3      53.5      115.5          5.490380         265.944000        950.356689
4      54.5      -62.5          7.143635         365.173126       1312.962646


In [12]:
import xarray as xr
import numpy as np
import pandas as pd

# Path to GLDAS VIC NetCDF file
file_path = 'D:\Groundwater Vulnerability Mapping\data\RAW Data\GLDAS_VIC10_M_2.1-20250814_153905\GLDAS_VIC10_M.A200001.021.nc4'  # Change this to your actual file path
ds = xr.open_dataset(file_path)

# Define target variables
variables = [
    'SoilMoi0_30cm_inst',
    'SoilMoi_depth2_inst',
    'SoilMoi_depth3_inst',
    'RootMoist_inst'
]

# Check all variables exist
for var in variables:
    if var not in ds:
        raise ValueError(f"Variable '{var}' not found in dataset.")

# Use the first time step
time_step = ds['time'][0]

# Create a mask of valid data (non-NaN) using one variable
valid_mask = ~np.isnan(ds['SoilMoi0_30cm_inst'].sel(time=time_step))

# Get valid (lat_idx, lon_idx) indices
valid_points = np.array(np.where(valid_mask.values)).T

# Ensure there are enough valid points
if len(valid_points) < 5:
    raise ValueError("Not enough valid data points in the dataset.")

# Randomly select 5 valid points
np.random.seed(42)
selected_indices = valid_points[np.random.choice(len(valid_points), 5, replace=False)]

# Extract coordinates
lats = ds['lat'].values
lons = ds['lon'].values

# Collect data
data = []
for idx in selected_indices:
    lat_idx, lon_idx = idx
    lat = lats[lat_idx]
    lon = lons[lon_idx]

    record = {'Latitude': lat, 'Longitude': lon}
    for var in variables:
        value = ds[var].sel(lat=lat, lon=lon, time=time_step, method='nearest').item()
        record[var] = value

    data.append(record)

# Create a DataFrame and print
df = pd.DataFrame(data)
print(df)


   Latitude  Longitude  SoilMoi0_30cm_inst  SoilMoi_depth2_inst  \
0      65.5     -149.5           84.679436           138.185181   
1      54.5      -84.5          113.104263           143.242996   
2      22.5       96.5           56.632904           262.413757   
3      53.5      115.5           85.961090           122.601730   
4      54.5      -62.5          121.322533           105.368515   

   SoilMoi_depth3_inst  RootMoist_inst  
0            21.403254      244.212296  
1            18.197269      274.562775  
2            22.713724      319.046692  
3            22.672068      231.266907  
4            24.721439      251.402374  


In [17]:
import pandas as pd

# Load one of your final GWS files (either an individual one or an ensemble one)
df = pd.read_csv('grace_processed_clipped_cm.csv')

# Check if the main GRACE column has any NaN (missing) values
missing_values_exist = df['lwe_thickness_cm'].isna().any()

if missing_values_exist:
    print("Diagnosis: NaN values ARE present. The issue might be with the plotting code.")
else:
    print("Diagnosis: No NaN values found. This confirms the data was interpolated at an earlier stage.")

Diagnosis: No NaN values found. This confirms the data was interpolated at an earlier stage.


In [18]:
import pandas as pd

# Load the final results file
df = pd.read_csv('NOAH_GWS_Final_Results_per_Pixel_cm.csv')
df['time'] = pd.to_datetime(df['time'])

# Create a complete monthly date range for the period
expected_range = pd.date_range(start=df['time'].min(), end=df['time'].max(), freq='MS')

# Find which months, if any, are missing from your data's time column
# We check against the unique values in the 'time' column
missing_months = expected_range.difference(df['time'].unique())

if len(missing_months) > 0:
    print("DIAGNOSIS: There ARE gaps in your time series. The problem might be with the plotting.")
    print("Missing months found:", missing_months)
else:
    print("DIAGNOSIS: There are NO gaps in your time series.")
    print("This confirms the source GRACE NetCDF file is a continuous, gap-filled dataset.")

DIAGNOSIS: There ARE gaps in your time series. The problem might be with the plotting.
Missing months found: DatetimeIndex(['2002-05-01 12:00:00', '2002-06-01 12:00:00',
               '2002-07-01 12:00:00', '2002-08-01 12:00:00',
               '2002-09-01 12:00:00', '2002-10-01 12:00:00',
               '2002-11-01 12:00:00', '2002-12-01 12:00:00',
               '2003-01-01 12:00:00', '2003-02-01 12:00:00',
               ...
               '2024-08-01 12:00:00', '2024-09-01 12:00:00',
               '2024-10-01 12:00:00', '2024-11-01 12:00:00',
               '2024-12-01 12:00:00', '2025-01-01 12:00:00',
               '2025-02-01 12:00:00', '2025-03-01 12:00:00',
               '2025-04-01 12:00:00', '2025-05-01 12:00:00'],
              dtype='datetime64[ns]', length=276, freq=None)


In [22]:
import xarray as xr
import pandas as pd

# --- Step 1: Define paths and the bounding box for your study area ---
grace_single_file_path = r"D:\Groundwater Vulnerability Mapping\data\RAW Data\GRACE\GRCTellus.JPL.200204_202505.GLO.RL06.3M.MSCNv04CRI.nc"

# Define the approximate lat/lon bounds for West Bengal
lat_min, lat_max = 21, 28
lon_min, lon_max = 85, 90

print("Attempting memory-efficient read of the single GRACE file...")

try:
    # --- Step 2: Open the dataset ---
    with xr.open_dataset(grace_single_file_path) as ds:
        
        # --- NEW: Print coordinate info to see the range ---
        print("\n--- DATASET COORDINATES ---")
        print(f"Latitude range: {ds['lat'].min().item()} to {ds['lat'].max().item()}")
        print(f"Longitude range: {ds['lon'].min().item()} to {ds['lon'].max().item()}")
        print("---------------------------\n")

        # --- Step 3: Spatially subset the data FIRST ---
        print("Subsetting data to the study area boundary...")
        # THE FIX: Use the correct slice order for the ascending latitude
        grace_subset_xr = ds.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))
        
        # --- Step 4: Convert the small subset to a DataFrame ---
        print("Converting the small subset to a DataFrame...")
        grace_df = grace_subset_xr.to_dataframe().reset_index()
    
    print("\n✅ SUCCESS: Data loaded into DataFrame without memory error.")
    
    # --- Step 5: VERIFY the time steps ---
    unique_months = grace_df['time'].nunique()
    min_date = grace_df['time'].min()
    max_date = grace_df['time'].max()
    print(f"\nDIAGNOSIS: Found {unique_months} unique months in the data.")
    print(f"Time range found: {min_date.date()} to {max_date.date()}")

except Exception as e:
    print(f"\nAn error occurred: {e}")

Attempting memory-efficient read of the single GRACE file...

--- DATASET COORDINATES ---
Latitude range: -89.75 to 89.75
Longitude range: 0.25 to 359.75
---------------------------

Subsetting data to the study area boundary...
Converting the small subset to a DataFrame...

✅ SUCCESS: Data loaded into DataFrame without memory error.

DIAGNOSIS: Found 245 unique months in the data.
Time range found: 2002-04-17 to 2025-05-16


In [ ]:
import pandas as pd

# --- Step 1: Load your final results file (which has gaps) ---
gappy_df = pd.read_csv('NOAH_GWS_Final_Results_per_Pixel_cm.csv')
gappy_df['time'] = pd.to_datetime(gappy_df['time'])

# --- Step 2: Standardize the dates to the 1st of the month ---
# We do this first to prepare for the merge
gappy_df['time'] = gappy_df['time'].dt.to_period('M').dt.start_time

# --- Step 3: Prepare the components for the master grid ---
# Get the full, continuous time range
full_date_range = pd.date_range(
    start=gappy_df['time'].min(), 
    end=gappy_df['time'].max(), 
    freq='MS'
)
full_date_df = pd.DataFrame({'time': full_date_range})

# Get all unique pixel coordinates
unique_pixels = gappy_df[['lat', 'lon']].drop_duplicates()

# --- Step 4: Build the complete master grid (scaffold) ---
# This uses a cross join to create a row for every pixel for every month
unique_pixels['key'] = 1
full_date_df['key'] = 1
scaffold_df = pd.merge(full_date_df, unique_pixels, on='key').drop('key', axis=1)

# --- Step 5: Merge the gappy data onto the full grid ---
# This step adds the data, leaving NaN where months were missing
final_gap_filled_df = pd.merge(scaffold_df, gappy_df, on=['time', 'lat', 'lon'], how='left')

# --- Step 6: Save the new, gap-filled file ---
output_filename = 'GWS_All_Pixels_All_Vars_Gap_Filled.csv'
final_gap_filled_df.to_csv(output_filename, index=False)

print(f"✅ Successfully created '{output_filename}'")
print("\n--- Preview of the new gap-filled data ---")
# Let's check a month that we know is a gap (June 2002)
# All variable columns should be NaN for this date.
print(final_gap_filled_df.head())